In [1]:
from fastai.vision.all import *
import fastai
import timm
import pandas as pd
from fastai.metrics import mae,rmse,mse

In [2]:
import torch
torch.cuda.set_device(3)

In [3]:
df = pd.read_csv('../dataset.csv')

In [4]:
df = df.drop(df[df.masa<0].index)

In [5]:
dftrain = df[(df.split=='train') | (df.split=='valid')]
dftest = df[(df.split=='test')]

In [6]:
def get_masa(f):
    return float(df[df['image_id']=='datasetClean/'+f.name]['masa'])

def get_volumen(f):
    return float(df[df['image_id']=='datasetClean/'+f.name]['volumen'])

def is_valid(f):
    return (df[df['image_id']=='datasetClean/'+f.name]['split']).values[0]=='valid'

In [7]:
def is_test(f):
    return (df[df['image_id']=='datasetClean/'+f.name]['split']).values[0]=='test'

In [8]:
dbtest = DataBlock(blocks = (ImageBlock, RegressionBlock()),
                 get_items=get_image_files, 
                 splitter=FuncSplitter(is_test),
                 get_y=get_volumen,
                 item_tfms = Resize(224),
                 batch_tfms=[*aug_transforms(size=224,flip_vert=True,
                                            max_zoom=1.0), Normalize.from_stats(*imagenet_stats)])

In [9]:
path = Path('../datasetAltura2/')
trainPath = path/'train'

In [10]:
dlstest = dbtest.dataloaders(path,bs=8)

In [11]:
learn1 = Learner(dlstest, timm.create_model('efficientnetv2_rw_s',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=[mae,rmse,mse]).to_fp16()

In [12]:
learn1.load('efficientnetv2_rw_s_volumen_height')

/home/jonathan/.virtualenvs/segmentation/lib/python3.6/site-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [13]:
preds1 = learn1.get_preds()

In [14]:
preds1TTA = learn1.tta()

In [15]:
dbtest = DataBlock(blocks = (ImageBlock, RegressionBlock()),
                 get_items=get_image_files, 
                 splitter=FuncSplitter(is_test),
                 get_y=get_volumen,
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=512,flip_vert=True,
                                            max_zoom=1.0), Normalize.from_stats(*imagenet_stats)])

In [16]:
path = Path('../datasetClean/')

In [17]:
dlstest = dbtest.dataloaders(path,bs=8)

In [18]:
learn2 = Learner(dlstest, timm.create_model('efficientnetv2_rw_s',num_classes=1,pretrained=True),
                loss_func=MSELossFlat(), metrics=mae).to_fp16()

In [19]:
learn2.load('efficientnetv2_rw_s_Volumen')

In [20]:
preds2 = learn2.get_preds()

In [21]:
preds2TTA = learn2.tta()

In [22]:
mae((preds1[0]+preds2[0])/2,preds1[1])

TensorBase(0.2927)

In [23]:
mae((preds1TTA[0]+preds2TTA[0])/2,preds1[1])

TensorBase(0.2849)